In [44]:
import pandas as pd
import numpy as np
import os
from cassandra.cluster import Cluster
from datetime import datetime
import plotly.graph_objects as go

In [45]:
# เชื่อมต่อกับ Cassandra Docker
cluster = Cluster(['127.0.0.1'], port=9042)
session = cluster.connect()

In [46]:
keyspace_name = "stock_data"
session.set_keyspace(keyspace_name)
table_name = "stock_price"
rows = session.execute(f"SELECT * FROM {table_name}")
df = pd.DataFrame(rows)
df.shape

(186, 7)

In [47]:
df.head()

,symbol,date,close,high,low,open,volume
0,VCOM,1970-01-01,2.40,2.44,2.38,2.44,152327
1,VGI,1970-01-01,2.22,2.28,2.20,2.26,62623507
2,ASP,1970-01-01,2.02,2.04,1.97,2.04,771843
3,ADVICE,1970-01-01,4.78,4.86,4.64,4.86,3628808
4,AYUD,1970-01-01,31.75,31.75,31.50,31.75,10733


In [48]:
rows_symbol = session.execute(f"SELECT * FROM stock_data.stock_price WHERE symbol = 'AOT'")
df_symbol = pd.DataFrame(rows_symbol)
df_symbol.head()

,symbol,date,close,high,low,open,volume
0,AOT,1970-01-01,39.0,39.25,38.25,39.0,27097430


In [49]:
row = df_symbol.iloc[0]
non_null = row[row.notnull()]
print(non_null)

symbol           AOT
date      1970-01-01
close           39.0
high           39.25
low            38.25
open            39.0
volume      27097430
Name: 0, dtype: object


In [50]:
# ดึงข้อมูลเฉพาะคอลัมน์ที่ต้องการจาก Cassandra
rows_nonnull = session.execute(
    """
    SELECT symbol, date, open, high, low, close, volume
    FROM stock_data.stock_price
    """
)

# แปลงเป็น DataFrame
df_nonnull = pd.DataFrame(rows_nonnull.all())  # ใช้ .all() เพื่อดึงผลลัพธ์ทั้งหมด

# แสดงผลลัพธ์
df_nonnull.head()


,symbol,date,open,high,low,close,volume
0,VCOM,1970-01-01,2.44,2.44,2.38,2.40,152327
1,VGI,1970-01-01,2.26,2.28,2.20,2.22,62623507
2,ASP,1970-01-01,2.04,2.04,1.97,2.02,771843
3,ADVICE,1970-01-01,4.86,4.86,4.64,4.78,3628808
4,AYUD,1970-01-01,31.75,31.75,31.50,31.75,10733


In [51]:
df_nonnull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  186 non-null    object 
 1   date    186 non-null    object 
 2   open    186 non-null    float64
 3   high    186 non-null    float64
 4   low     186 non-null    float64
 5   close   186 non-null    float64
 6   volume  186 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 10.3+ KB


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   symbol  186 non-null    object 
 1   date    186 non-null    object 
 2   close   186 non-null    float64
 3   high    186 non-null    float64
 4   low     186 non-null    float64
 5   open    186 non-null    float64
 6   volume  186 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 10.3+ KB


In [53]:
df.isnull().sum()

symbol    0
date      0
close     0
high      0
low       0
open      0
volume    0
dtype: int64